# Задача: Восстановление пропущенных пробелов в тексте с помощью NLP / DL / алгоритма.

## Описание окружения и подготовки

### Используемое окружение
Работа выполнялась в Kaggle-ноутбуке с использованием ускорителя **GPU T4 x2** для повышения производительности вычислений. Это позволило ускорить обучение модели и обработку данных.

### Установка и импорт библиотек
Для выполнения задачи были установлены и импортированы необходимые библиотеки. Установка выполнялась в Kaggle-ноутбуке с использованием команды `!pip install`, а импорт — через стандартный синтаксис Python.

In [1]:
!pip install Wikipedia-API

  Preparing metadata (setup.py) ... done
  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=9a046c3cd6764fa2a181f51dc8d3e1911f51d8ac97c36b457d689770f3c974b2
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built Wikipedia-API


In [2]:
import logging
from logging.handlers import RotatingFileHandler
import random
import torch
import time
import re
import wikipediaapi
from uuid import uuid4
from datasets import Dataset, DatasetDict
from transformers import ByT5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, T5Config
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd

2025-09-22 11:47:48.640138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758541668.961083      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758541669.054217      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Настройка логгера в Python с использованием модуля `logging`

Этот код настраивает систему логирования для записи сообщений в консоль и файл с использованием Python-модуля `logging`. Логгер используется для отслеживания событий во время выполнения программы, для вывода информации о процессе обучения модели.

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.handlers.clear()  

if not logger.handlers:
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    log_file = 'word_segmentation.log'
    file_handler = RotatingFileHandler(log_file, maxBytes=10*1024*1024, backupCount=5, encoding='utf-8')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    logger.info("Логгер успешно настроен")

2025-09-22 11:48:04 - __main__ - INFO - Логгер успешно настроен


## Настройка воспроизводимости результатов и логирование

Этот код выполняет инициализацию генераторов случайных чисел для обеспечения воспроизводимости результатов в Python-программе, а также записывает соответствующее сообщение в лог.

In [4]:
random.seed(42)
torch.manual_seed(42)
logger.info("Инициализированы случайные seed для воспроизводимости результатов")

2025-09-22 11:48:04 - __main__ - INFO - Инициализированы случайные seed для воспроизводимости результатов


## Описание функции `fetch_wiki_texts`

Функция `fetch_wiki_texts` предназначена для загрузки текстовых фраз из Википедии на указанном языке, с определёнными критериями длины и случайными модификациями. Она используется для создания набора данных. Функция активно использует логирование для отслеживания процесса.

### Параметры функции
- **lang**: Код языка (например, `"en"` для английского, `"ru"` для русского).
- **num_pages**: Количество фраз, которые нужно извлечь (по умолчанию 100).

In [5]:
def fetch_wiki_texts(lang, num_pages=100):
    logger.info(f"Загружаем {num_pages} коротких фраз из Википедии на языке: {lang}")
    start_time = time.time()
    
    #Создание объекта Википедии
    
    wiki = wikipediaapi.Wikipedia(
        language=lang,
        user_agent=f"WordSegmentationBot/1.0 (https://x.ai; {uuid4()})"
    )
    
    #Список заголовков страниц
    
    page_titles = {
        "en": [
            "Smartphone", "Laptop", "Coffee", "Chocolate", "Bicycle",
            "Book", "Car", "Furniture", "Clothing", "Shoes",
            "Graphics card", "Television", "Headphones", "Camera", "Tablet",
            "Main Street", "Park Avenue", "Broadway", "Oxford Street", "Fifth Avenue",
            "Times Square", "Trafalgar Square", "Piccadilly Circus", "Champs-Élysées", "Alexanderplatz",
            "Shopping", "Restaurant", "Supermarket", "Bank", "Hospital",
            "School", "University", "Hotel", "Airport", "Station",
            "Market", "Lawyer", "Contract", "Warehouse", "Office"
        ],
        "ru": [
            "Смартфон", "Ноутбук", "Кофе", "Шоколад", "Велосипед",
            "Книга", "Автомобиль", "Мебель", "Одежда", "Обувь",
            "Видеокарта", "Телевизор", "Наушники", "Камера", "Планшет",
            "Тверская улица", "Невский проспект", "Арбат", "Кутузовский проспект", "Садовое кольцо",
            "Красная площадь", "Манежная площадь", "Улица Горького", "Проспект Мира", "Ленинский проспект",
            "Магазин", "Ресторан", "Супермаркет", "Банк", "Больница",
            "Школа", "Университет", "Гостиница", "Аэропорт", "Вокзал",
            "Рынок", "Юрист", "Договор", "Склад", "Офис",  "iPhone","Бытовая техника", "Собака", 
            "Кровать", "Врач", "Стоматолог", "Мебель", "Московский метрополитен", "Рюкзак", 
            "Скейтборд", "Английский язык", "Ремонт", "Химия", "Парикмахер", "Автомобиль", 
            "Программирование", "Солнце", "Календарь", "Песня", "Чай", "Лампа накаливания", 
            "Герой", "Литература", "Смартфон", "Кофе", "Телевизор", "Университет", "Рынок", "Офис"
        ]
    }.get(lang, [
        "Computer", "Phone", "Food", "City", "Street",
        "Shop", "Market", "Bank", "School", "Transport"
    ])
    
    #Расширение списка заголовков
    
    page_titles = (page_titles * (num_pages // len(page_titles) + 1))[:num_pages * 2]
    
    #Извлечение текстов
    
    texts = []
    successful = 0
    for i, title in enumerate(page_titles, 1):
        try:
            page = wiki.page(title)
            if page.exists():
                content = page.text
                sentences = re.split(r'[.!?]+', content)
                
                #Фильтрация и модификация предложений
                
                for sentence in sentences:
                    clean = ' '.join(sentence.split()).strip()
                    if  30<= len(clean) <=50:
                        if random.random() < 0.2: 
                            words = clean.split()
                            if len(words) > 1:
                                pos = random.randint(1, len(words) - 1)
                                clean = ' '.join(words[:pos]) + ', ' + ' '.join(words[pos:])
                        if random.random() < 0.2:  
                            clean += ' ' + str(random.randint(100, 300))
                        texts.append(clean)
                        successful += 1
                        logger.debug(f"Извлечена фраза из {title}: {clean} (длина: {len(clean)})")
                        
                    #Обработка ошибок и пропуск страниц
                    
                    if successful >= num_pages:
                        break
                else:
                    logger.debug(f"Пропущена страница {title}: нет подходящих фраз")
            else:
                logger.debug(f"Пропущена страница {title}: не существует")
        except Exception as e:
            logger.warning(f"Ошибка при обработке страницы {title}: {str(e)}")
        if successful >= num_pages:
            break
            
    #Дополнение недостающих фраз
    
    if len(texts) < num_pages:
        logger.warning(f"Извлечено только {len(texts)} фраз вместо {num_pages}. Дополняем...")
        while len(texts) < num_pages:
            texts.append(random.choice(texts) if texts else f"Sample text {lang} {len(texts)}")
    
    logger.info(f"Извлечено {len(texts)} фраз на {lang} за {time.time() - start_time:.2f} секунд")
    return texts[:num_pages]

## Описание функции `generate_data`

Функция `generate_data` создаёт датасет для задачи сегментации слов, комбинируя тексты на английском и русском языках, полученные из Википедии с помощью функции `fetch_wiki_texts`. Она формирует пары входных и выходных данных, где вход — текст без пробелов и запятых, а выход — исходный текст с пробелами.

### Параметры функции
- **num_samples**: Количество сгенерированных примеров (по умолчанию 5000).

In [6]:
def generate_data(num_samples=5000):
    logger.info("Подготовка датасета")

    #Получение текстов из Википедии
    
    en_texts = fetch_wiki_texts('en', num_samples // 2)
    ru_texts = fetch_wiki_texts('ru', num_samples // 2)
    
    if not en_texts or not ru_texts:
        logger.error("Не удалось получить тексты из Википедии")
        raise ValueError("Не удалось получить тексты из Википедии")

    #Генерация датасета
    
    data = []
    for _ in range(num_samples):
        if random.random() < 0.3:  # 30% mixed
            en_sent = random.choice(en_texts)
            ru_sent = random.choice(ru_texts)
            original = en_sent + ' ' + ru_sent
        elif random.random() < 0.5:
            original = random.choice(en_texts)
        else:
            original = random.choice(ru_texts)

        #Формирование входных и выходных данных
        
        no_space = ''.join(original.replace(',', '').replace(' ', ''))
        if len(no_space) <= 512:  # Проверка длины
            input_text = f"insert spaces: {no_space}"
            data.append({"input": input_text, "output": original})
    
    return data

## Описание процесса подготовки датасета

Этот код выполняет подготовку датасета для машинного обучения, используя функцию `generate_data` (описанную ранее), преобразует данные в объект `Dataset` из библиотеки Hugging Face `datasets`, разделяет его на обучающую и валидационную выборки, и логирует результаты. 

In [7]:
logger.info("Подготовка датасета")
start_time = time.time()
data = generate_data(12000) 
dataset = Dataset.from_list(data)
split = dataset.train_test_split(test_size=0.1)
dataset_dict = DatasetDict({"train": split["train"], "validation": split["test"]})
logger.info(f"Датасет подготовлен: {len(dataset_dict['train'])} обучающих, {len(dataset_dict['validation'])} валидационных примеров за {time.time() - start_time:.2f} секунд")

2025-09-22 11:48:04 - __main__ - INFO - Подготовка датасета
2025-09-22 11:48:04 - __main__ - INFO - Подготовка датасета
2025-09-22 11:48:04 - __main__ - INFO - Загружаем 6000 коротких фраз из Википедии на языке: en
2025-09-22 11:49:16 - __main__ - INFO - Извлечено 6000 фраз на en за 72.21 секунд
2025-09-22 11:49:16 - __main__ - INFO - Загружаем 6000 коротких фраз из Википедии на языке: ru
2025-09-22 11:50:15 - __main__ - INFO - Извлечено 6000 фраз на ru за 58.65 секунд
2025-09-22 11:50:15 - __main__ - INFO - Датасет подготовлен: 10800 обучающих, 1200 валидационных примеров за 130.97 секунд


## Описание процесса загрузки модели и токенизатора

Этот код выполняет загрузку предобученной модели `google/byt5-small` и её токенизатора из библиотеки Hugging Face `transformers` для задачи обработки текста, например, сегментации слов. Процесс сопровождается логированием времени выполнения. 

## Выбор модели `google/byt5-small`

Для задачи сегментации слов была выбрана модель `google/byt5-small` из-за её компактного размера, что делает её подходящей для работы в Kaggle-ноутбуке с ускорителем GPU T4 x2 (16 ГБ памяти). Модель `google/byt5-base`, имеющая больший размер, не использовалась из-за ограничений по памяти GPU, которые приводили к ошибкам нехватки памяти при загрузке и обучении.

### Причины выбора
- **Меньший размер**: `byt5-small` требует меньше памяти GPU, что позволяет эффективно выполнять обучение и инференс на доступном оборудовании.
- **Ограничения Kaggle**: GPU T4 x2 имеет ограниченный объём памяти, и более крупная модель `byt5-base` вызывала ошибки типа `CUDA out of memory`.

In [8]:
model_name = "google/byt5-small"
logger.info(f"Загружаем модель и токенизатор: {model_name}")
start_time = time.time()
tokenizer = ByT5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
logger.info(f"Модель и токенизатор загружены за {time.time() - start_time:.2f} секунд")

2025-09-22 11:50:15 - __main__ - INFO - Загружаем модель и токенизатор: google/byt5-small


tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

2025-09-22 11:50:24 - __main__ - INFO - Модель и токенизатор загружены за 9.71 секунд


## Описание функции `preprocess`

Функция `preprocess` выполняет токенизацию входных и выходных данных датасета для подготовки к обучению модели `google/byt5-small`. Она используется для обработки текстов, созданных функцией `generate_data`, в формате, подходящем для задачи сегментации слов. 

### Параметры функции
- **examples**: Словарь (или объект `Dataset` из библиотеки `datasets`), содержащий ключи `"input"` и `"output"`.
  - `"input"`: Текст с префиксом `"insert spaces: ..."`, например, `"insert spaces: Helloworld"`.
  - `"output"`: Исходный текст с пробелами, например, `"Hello world"`.

In [9]:
def preprocess(examples):
    inputs = tokenizer(examples["input"], max_length=256, truncation=True)
    labels = tokenizer(examples["output"], max_length=256, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

## Описание процесса токенизации датасета

Этот код выполняет токенизацию датасета, созданного ранее, используя функцию `preprocess` для подготовки данных к обучению модели `google/byt5-small`.

In [10]:
logger.info("Токенизация датасета")
start_time = time.time()
tokenized_datasets = dataset_dict.map(preprocess, batched=True)
logger.info(f"Датасет токенизирован за {time.time() - start_time:.2f} секунд")

2025-09-22 11:50:24 - __main__ - INFO - Токенизация датасета


Map:   0%|          | 0/10800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

2025-09-22 11:50:31 - __main__ - INFO - Датасет токенизирован за 6.80 секунд


## Описание процесса очистки датасета и логирования

Этот код выполняет удаление ненужных колонок (`"input"` и `"output"`) из токенизированного датасета, созданного ранее, и логирует изменения.

Эти колонки содержат исходные текстовые данные (например, "insert spaces: Helloworld" и "Hello world"), которые больше не нужны после токенизации, так как данные уже преобразованы в `input_ids`, `attention_mask` и `labels`.

In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(["input", "output"])
logger.info("Удалены ненужные колонки 'input' и 'output' из датасета")
logger.info(f"Обновлённые колонки train: {tokenized_datasets['train'].column_names}")
tokenized_datasets

2025-09-22 11:50:31 - __main__ - INFO - Удалены ненужные колонки 'input' и 'output' из датасета
2025-09-22 11:50:31 - __main__ - INFO - Обновлённые колонки train: ['input_ids', 'attention_mask', 'labels']


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10800
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1200
    })
})

## Описание инициализации Data Collator

Этот код инициализирует объект `DataCollatorForSeq2Seq` из библиотеки Hugging Face `transformers` для подготовки данных к обучению модели `google/byt5-small` в задаче последовательностного преобразования (sequence-to-sequence).

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
logger.info("Data collator инициализирован")

2025-09-22 11:50:37 - __main__ - INFO - Data collator инициализирован


## Описание процесса обучения модели

Этот код выполняет обучение модели `google/byt5-small`. Он включает настройку загрузчиков данных, оптимизатора, планировщика скорости обучения, обучение с накоплением градиентов, периодическую валидацию и сохранение модели. Процесс сопровождается подробным логированием. 

### Параметры обучения
- **EPOCHS**: `10` — количество эпох обучения.
- **BATCH_SIZE**: `16` — размер батча.
- **GRADIENT_ACCUMULATION_STEPS**: `2` — количество шагов накопления градиентов (эффективный размер батча = `BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS` = 32).
- **LEARNING_RATE**: `2e-5` — скорость обучения.
- **LOG_STEPS**: `50` — частота логирования метрик во время обучения.
- **EVAL_STEPS**: `200` — частота выполнения валидации.
- **WARMUP_STEPS**: `100` шагов  для постепенного увеличения скорости обучения.

In [13]:
import torch
torch.cuda.empty_cache()
logger.info("Очищена кэшированная память GPU")

2025-09-22 11:50:38 - __main__ - INFO - Очищена кэшированная память GPU


In [15]:
EPOCHS = 10 
BATCH_SIZE = 16  
GRADIENT_ACCUMULATION_STEPS = 2  
LEARNING_RATE = 2e-5  
WARMUP_STEPS = 100 
LOG_STEPS = 50
EVAL_STEPS = 200

#Создание загрузчиков данных

train_dataloader = DataLoader(
    tokenized_datasets['train'], 
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator,
    num_workers=2,  
    pin_memory=True  
)

eval_dataloader = DataLoader(
    tokenized_datasets['validation'], 
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    num_workers=2,
    pin_memory=True
)

#Настройка оптимизатора

optimizer = optim.AdamW(
    model.parameters(), 
    lr=LEARNING_RATE,
    weight_decay=0.01,  
    eps=1e-8 
)

#Настройка планировщика скорости обучения

num_training_steps = len(train_dataloader) * EPOCHS
num_warmup_steps = WARMUP_STEPS

scheduler = optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda=lambda step: min(1.0, step / num_warmup_steps) if step < num_warmup_steps 
    else max(0.0, (num_training_steps - step) / (num_training_steps - num_warmup_steps))
)

#Логирование гиперпараметров

logger.info(f"   Эпохи: {EPOCHS}")
logger.info(f"   Batch size: {BATCH_SIZE} (accumulation: {GRADIENT_ACCUMULATION_STEPS})")
logger.info(f"   Effective batch: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
logger.info(f"   Learning rate: {LEARNING_RATE}")
logger.info(f"   Warmup steps: {num_warmup_steps}")
logger.info(f"   Всего шагов: {num_training_steps}")
logger.info("=" * 60)

#Функция вычисления точности

def compute_accuracy(logits, labels):
    predictions = torch.argmax(logits, dim=-1)
    mask = labels != -100  
    correct = (predictions == labels) & mask
    accuracy = correct.sum().float() / mask.sum().float()
    return accuracy.item()

#Подготовка модели

model.train()
model.to('cuda') 

#Цикл обучения

global_step = 0
best_eval_loss = float('inf')
total_loss = 0
gradient_accumulation_counter = 0

#Внешний цикл по эпохам

for epoch in range(EPOCHS):
    logger.info(f"ЭПОХА {epoch + 1}/{EPOCHS} НАЧАЛАСЬ")
    epoch_loss = 0
    epoch_accuracy = 0

    #Внутренний цикл по батчам
    
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to('cuda') for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss

        #Нормировка потерь для накопления градиентов
        
        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        #Обратное распространение
        
        loss.backward()
        
        gradient_accumulation_counter += 1
        total_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        epoch_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS

        #Вычисление точности
        
        if hasattr(outputs, 'logits'):
            accuracy = compute_accuracy(outputs.logits, batch['labels'])
            epoch_accuracy += accuracy

        #Обновление параметров модели
        
        if gradient_accumulation_counter % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
            global_step += 1

            #Логирование каждые LOG_STEPS
            
            if global_step % LOG_STEPS == 0:
                avg_loss = total_loss / LOG_STEPS
                current_lr = scheduler.get_last_lr()[0]
                logger.info(f"Шаг {global_step:4d} | Эпоха {epoch + 1} | "
                           f"Loss: {loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f} | "
                           f"Avg: {avg_loss:.4f} | LR: {current_lr:.2e} | "
                           f"Accuracy: {accuracy:.3f}")
                total_loss = 0
                
            #Валидация
            
            if global_step % EVAL_STEPS == 0:
                logger.info(f"Валидация на шаге {global_step}...")
                model.eval()
                eval_loss = 0
                eval_accuracy = 0
                eval_steps = 0
                
                with torch.no_grad():
                    for eval_batch in eval_dataloader:
                        eval_batch = {k: v.to('cuda') for k, v in eval_batch.items()}
                        eval_outputs = model(**eval_batch)
                        eval_loss += eval_outputs.loss.item()
                        
                        if hasattr(eval_outputs, 'logits'):
                            eval_accuracy += compute_accuracy(eval_outputs.logits, eval_batch['labels'])
                        
                        eval_steps += 1
                        
                        if eval_steps >= 100:  
                            break
                
                avg_eval_loss = eval_loss / eval_steps
                avg_eval_accuracy = eval_accuracy / eval_steps
                
                logger.info(f"Validation Loss: {avg_eval_loss:.4f} | Accuracy: {avg_eval_accuracy:.3f}")
                
                if avg_eval_loss < best_eval_loss:
                    best_eval_loss = avg_eval_loss
                    logger.info(f"Лучшее значение! Best Eval Loss: {best_eval_loss:.4f}")
                    
                
                model.train()

    #Итоги эпохи
    
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    avg_epoch_accuracy = epoch_accuracy / len(train_dataloader)
    
    logger.info(f"ЭПОХА {epoch + 1} ЗАВЕРШЕНА | "
               f"Средний Loss: {avg_epoch_loss:.4f} | "
               f"Accuracy: {avg_epoch_accuracy:.3f}")
    logger.info("-" * 40)

#Сохранение модели

state_dict = model.module.state_dict() if isinstance(model, torch.nn.DataParallel) else model.state_dict()
torch.save({
    'model_state_dict': state_dict,
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'best_loss': best_eval_loss,
    'final_epoch': EPOCHS,
}, "final_model.pt")
tokenizer.save_pretrained("final_model")
logger.info("Финальная модель и токенизатор сохранены в 'final_model'")

logger.info("=" * 60)
logger.info(f"Обучение завершено!")
logger.info(f"Лучший Eval Loss: {best_eval_loss:.4f}")
logger.info("=" * 60)

2025-09-22 11:51:22 - __main__ - INFO -    Эпохи: 10
2025-09-22 11:51:22 - __main__ - INFO -    Batch size: 16 (accumulation: 2)
2025-09-22 11:51:22 - __main__ - INFO -    Effective batch: 32
2025-09-22 11:51:22 - __main__ - INFO -    Learning rate: 2e-05
2025-09-22 11:51:22 - __main__ - INFO -    Warmup steps: 100
2025-09-22 11:51:22 - __main__ - INFO -    Всего шагов: 6750
2025-09-22 11:51:22 - __main__ - INFO - ============================================================
2025-09-22 11:51:22 - __main__ - INFO - ЭПОХА 1/10 НАЧАЛАСЬ
2025-09-22 11:53:29 - __main__ - INFO - Шаг   50 | Эпоха 1 | Loss: 1.5650 | Avg: 3.4290 | LR: 1.00e-05 | Accuracy: 0.676
2025-09-22 11:55:38 - __main__ - INFO - Шаг  100 | Эпоха 1 | Loss: 0.9559 | Avg: 2.4416 | LR: 2.00e-05 | Accuracy: 0.781
2025-09-22 11:57:47 - __main__ - INFO - Шаг  150 | Эпоха 1 | Loss: 0.6057 | Avg: 1.4441 | LR: 1.98e-05 | Accuracy: 0.868
2025-09-22 11:59:57 - __main__ - INFO - Шаг  200 | Эпоха 1 | Loss: 0.3740 | Avg: 0.9345 | LR: 1.97

## Описание процесса чтения и парсинга тестового датасета

Этот код выполняет чтение тестового датасета из файла `dataset_1937770_3.txt`

In [16]:
file_path = '/kaggle/input/test-3/dataset_1937770_3.txt'

parsed_data = []
with open(file_path, 'r', encoding='utf-8') as file:
    
    #Чтение файла с обработкой строк
    
    for line_num, line in enumerate(file, start=0):
        line = line.strip()  
        if not line or line.startswith('id,'):  
            continue
            
        #Парсинг строк и обработка ошибок
        
        try:
            id_str, text = line.split(',', 1)  
            parsed_data.append({
                'id': int(id_str.strip()),  
                'text_no_spaces': text.strip()  
            })
        except ValueError as e:
            print(f"Ошибка в строке {line_num}: '{line}' — {e}")
            continue

df = pd.DataFrame(parsed_data)

## Описание функции `batch_insert_spaces`

Функция `batch_insert_spaces` выполняет пакетное предсказание для задачи сегментации слов, используя модель `google/byt5-small` и её сохранённые веса. Она принимает список текстов без пробелов, добавляет к ним префикс `"insert spaces: "`, токенизирует, выполняет инференс модели и декодирует результаты в тексты с пробелами.

### Параметры функции
- **texts_list**: Список строк без пробелов (например, `["Helloworld", "СмартфонКнига"]`).
- **model_weights_path**: Путь к сохранённым весам модели (по умолчанию `"./final_model.pt"`).
- **tokenizer_path**: Путь к сохранённому токенизатору (по умолчанию `"./final_model"`).
- **device**: Устройство для инференса (`"cuda"` или `"cpu"`; по умолчанию определяется автоматически).
- **batch_size**: Размер батча для обработки текстов (по умолчанию 8).

In [17]:
def batch_insert_spaces(texts_list, model_weights_path="./final_model.pt", 
                       tokenizer_path="./final_model", device=None, batch_size=8):
    
    logger.info(f"Запускаем пакетное предсказание для {len(texts_list)} текстов")
    start_time = time.time()
    
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        #Загрузка токенизатора
        logger.info(f"Загружаем токенизатор из {tokenizer_path}")
        tokenizer = ByT5Tokenizer.from_pretrained(tokenizer_path)
        
        #Создание конфигурации модели
        config = T5Config.from_pretrained('google/byt5-small')

        #Инициализация и загрузка модели
        
        model = T5ForConditionalGeneration(config)
        
        logger.info(f"Загружаем веса модели из {model_weights_path}")
        checkpoint = torch.load(model_weights_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        model.to(device)
        model.eval()

        #Инициализация списка результатов
        
        results = []
        
        #Пакетная обработка текстов
        logger.info(f"Обрабатываем файл")
        for i in range(0, len(texts_list), batch_size):
            batch_texts = texts_list[i:i + batch_size]
            batch_input_texts = [f"insert spaces: {text}" for text in batch_texts]
            
            #Токенизация батча
            
            inputs = tokenizer(
                batch_input_texts, 
                return_tensors="pt",
                max_length=512,
                truncation=True,
                padding=True,
                return_attention_mask=True
            ).to(device)

            #Инференс модели
            
            with torch.no_grad():
                outputs = model.generate(
                    inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_length=512,
                    num_beams=3,
                    early_stopping=True,
                    length_penalty=0.8
                )
                
            #Декодирование результатов
            
            batch_results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            results.extend(batch_results)
            
        #Логирование завершения
        
        execution_time = time.time() - start_time
        logger.info(f"Предсказание завершено за {execution_time:.2f} секунд")
        logger.info(f"Обработано {len(texts_list)} текстов, среднее время: {execution_time/len(texts_list):.3f} сек/текст")
        
        return results
        
    #Обработка ошибок
    
    except Exception as e:
        logger.error(f"Ошибка при пакетном предсказании: {str(e)}")
        raise

## Описание процесса предсказания и добавления результатов в DataFrame

Этот код выполняет предсказание восстановленных текстов (с пробелами) для тестового датасета, используя функцию `batch_insert_spaces`, и добавляет результаты в DataFrame `df` в новую колонку `"predicted_text"`. 

In [18]:
test_texts = df['text_no_spaces'].tolist()

restored_texts = batch_insert_spaces(test_texts)

df["predicted_text"] = restored_texts

2025-09-22 14:25:54 - __main__ - INFO - Запускаем пакетное предсказание для 1005 текстов
2025-09-22 14:25:54 - __main__ - INFO - Загружаем токенизатор из ./final_model
2025-09-22 14:25:59 - __main__ - INFO - Загружаем веса модели из ./final_model.pt
2025-09-22 14:26:02 - __main__ - INFO - Обрабатываем файл
2025-09-22 14:27:30 - __main__ - INFO - Предсказание завершено за 95.26 секунд
2025-09-22 14:27:30 - __main__ - INFO - Обработано 1005 текстов, среднее время: 0.095 сек/текст


## Описание функции `text_to_positions` и обработки DataFrame

Этот код выполняет преобразование предсказанных текстов с пробелами в список позиций пробелов относительно текста без пробелов и добавляет результаты в DataFrame `df` в виде новой колонки `"predicted_positions"`. Затем создаётся новый DataFrame `df_n`, содержащий только колонки `"id"` и `"predicted_positions"`. 

In [19]:
def text_to_positions(original_text, predicted_text):
    original_text = original_text.strip()
    predicted_text = predicted_text.strip()
    
    predicted_no_spaces = predicted_text.replace(" ", "")
    
    positions = []
    current_pos = 0
    
    for char in predicted_text:
        if char == ' ':
            positions.append(current_pos)
        else:
            current_pos += 1
    
    return str(positions)

df["predicted_positions"] = df.apply(
    lambda row: text_to_positions(row["text_no_spaces"], row["predicted_text"]), 
    axis=1
)
df_n = df[["id", "predicted_positions"]].copy()

## Описание сохранения результатов в CSV

Этот код сохраняет DataFrame `df_n`, содержащий колонки `"id"` и `"predicted_positions"`, в файл `task_data_with_positions.csv` в Kaggle-ноутбуке. Также выводится сообщение в консоль, подтверждающее успешное сохранение. 

In [20]:
df_n.to_csv("task_data_with_positions.csv", index=False, encoding="utf-8")
print("✅ Данные сохранены в task_data_with_positions.csv")

✅ Данные сохранены в task_data_with_positions.csv


In [21]:
df_n

,id,predicted_positions
0,0,"[5, 7, 10, 12]"
1,1,"[3, 6, 7, 10]"
2,2,"[4, 12, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[2, 5, 10]"
...,...,...
1000,1000,[3]
1001,1001,"[7, 10, 16]"
1002,1002,[19]
1003,1003,"[9, 20, 23]"
